In [1]:
import os

import fsspec
import geopandas as gpd

from utils.hls.catalog import HLSBand
from utils.hls.catalog import HLSCatalog
from utils.hls.catalog import HLSTileLookup
from utils.hls.catalog import fia_csv_to_data_catalog_input

In [2]:
os.environ['AZURE_ACCOUNT_KEY'] = ""

In [3]:
lookup = HLSTileLookup()

Reading tile extents...
Read tile extents for 56686 tiles


In [4]:
bands = [
    HLSBand.COASTAL_AEROSOL,
    HLSBand.BLUE,
    HLSBand.GREEN,
    HLSBand.RED,
    HLSBand.NIR_NARROW,
    HLSBand.SWIR1,
    HLSBand.SWIR2,
    HLSBand.CIRRUS,
    HLSBand.QA,
]

In [5]:
storage_container = 'fia'
account_name = 'usfs'
account_key = os.environ['AZURE_ACCOUNT_KEY']

In [ ]:
%%time

# 2015-2019 USA
geom = gpd.read_file('cb_2018_conus_20m.geojson').to_crs('EPSG:4326')
years = [2015, 2016, 2017, 2018, 2019]
conus_catalog = HLSCatalog.from_geom(geom, years, bands, lookup)

In [ ]:
path = 'catalogs/hls_conus_2015-2019'
write_store = fsspec.get_mapper(
    f"az://{storage_container}/{path}.zarr",
    account_name=account_name,
    account_key=account_key
)
conus_catalog.to_zarr(write_store)

In [6]:
# 2015-2019 Washington State
bbox = [-124.76074218749999, 45.44471679159555, -116.91650390625, 49.05227025601607]
years = [2015, 2016, 2017, 2018, 2019]
wa_catalog = HLSCatalog.from_bbox(bbox, years, bands, lookup)

In [7]:
path = 'catalogs/hls_wa_2015-2019'
write_store = fsspec.get_mapper(
    f"az://{storage_container}/{path}.zarr",
    account_name=account_name,
    account_key=account_key
)
wa_catalog.to_zarr(write_store)

In [6]:
# 2015-2019 Arizona

bbox = [-114.86206054687499, 31.306715155075167, -109.0283203125, 37.02886944696474]
years = [2015, 2016, 2017, 2018, 2019]
az_catalog = HLSCatalog.from_bbox(bbox, years, bands, lookup)

In [7]:
path = 'catalogs/hls_az_2015-2019'
write_store = fsspec.get_mapper(
    f"az://{storage_container}/{path}.zarr",
    account_name=account_name,
    account_key=account_key
)
az_catalog.to_zarr(write_store)

In [7]:
# 2015-2019 Western US (Montana/Wyoming/Colorado/New Mexico and west)
bbox = [-124.78, 31.33, -102.04, 49.02]
years = [2015, 2016, 2017, 2018, 2019]
west_catalog = HLSCatalog.from_bbox(bbox, years, bands, lookup)

In [8]:
path = 'catalogs/hls_west_2015-2019'
write_store = fsspec.get_mapper(
    f"az://{storage_container}/{path}.zarr",
    account_name=account_name,
    account_key=account_key
)
west_catalog.to_zarr(write_store)

In [6]:
bands = [
    HLSBand.COASTAL_AEROSOL,
    HLSBand.BLUE,
    HLSBand.GREEN,
    HLSBand.RED,
    HLSBand.NIR_NARROW,
    HLSBand.SWIR1,
    HLSBand.SWIR2,
]

df = fia_csv_to_data_catalog_input('./fia_no_pltcn.csv')
pt_catalog = HLSCatalog.from_point_pandas(df, bands, include_scenes=False)

Reading tile extents...
Read tile extents for 56686 tiles


In [7]:
write_store = fsspec.get_mapper(
    f"az://fia/catalogs/fia_tiles.zarr",
    account_name="usfs",
    account_key=os.environ['AZURE_ACCOUNT_KEY']
)
pt_catalog.to_zarr(write_store)